In [3]:
import requests
import json
from datetime import date, timedelta
import pandas as pd


class DownloadBaiDuIndex(object):
    def __init__(self, cookie):
        self.cookie = cookie
        self.headers = {
            "Connection": "keep-alive",
            "Accept": "application/json, text/plain, */*",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
            "Sec-Fetch-Site": "same-origin",
            "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Dest": "empty",
            "Referer": "https://index.baidu.com/v2/main/index.html",
            "Accept-Language": "zh-CN,zh;q=0.9",
            'Cookie': self.cookie,
            "Host": "index.baidu.com",
            "X-Requested-With": "XMLHttpRequest",
            "Cipher-Text": "1656572408684_1656582701256_Nvm1pABkNsfD7V9VhZSzzFiFKylr3l5NR3YDrmHmH9yfFicm+Z9kmmwKVqVV6unvzAEh5hgXmgelP+OyOeaK8F21LyRVX1BDjxm+ezsglwoe1yfp6lEpuvu5Iggg1dz3PLF8e2II0e80ocXeU0jQFBhSbnB2wjhKl57JggTej12CzuL+h9eeVWdaMO4DSBWU2XX6PfbN8pv9+cdfFhVRHCzb0BJBU3iccoFczwNQUvzLn0nZsu0YPtG5DxDkGlRlZrCfKMtqKAe1tXQhg3+Oww4N3CQUM+6A/tKZA7jfRE6CGTFetC7QQyKlD7nxabkQ5CReAhFYAFAVYJ+sEqmY5pke8s3+RZ6jR7ASOih6Afl35EArbJzzLpnNPgrPCHoJiDUlECJveul7P5vvXl/O/Q==",

        }

    def decrypt(self, ptbk, index_data):
        n = len(ptbk) // 2
        a = dict(zip(ptbk[:n], ptbk[n:]))
        return "".join([a[s] for s in index_data])

    def get_index_data_json(self, keys, start=None, end=None):
        words = [[{"name": key, "wordType": 1}] for key in keys]
        words = str(words).replace(" ", "").replace("'", "\"")

        url = f'http://index.baidu.com/api/SearchApi/index?area=0&word={words}&area=0&startDate={start}&endDate={end}'
        print(words, start, end)
        res = requests.get(url, headers=self.headers)
        data = res.json()['data']
        uniqid = data['uniqid']
        url = f'http://index.baidu.com/Interface/ptbk?uniqid={uniqid}'
        res = requests.get(url, headers=self.headers)
        ptbk = res.json()['data']
        result = {}
        result["startDate"] = start
        result["endDate"] = end
        for userIndexe in data['userIndexes']:
            name = userIndexe['word'][0]['name']
            tmp = {}
            index_all = userIndexe['all']['data']
            index_all_data = [int(e) for e in self.decrypt(ptbk, index_all).split(",")]
            tmp["all"] = index_all_data
            index_pc = userIndexe['pc']['data']
            index_pc_data = [int(e) for e in self.decrypt(ptbk, index_pc).split(",")]
            tmp["pc"] = index_pc_data
            index_wise = userIndexe['wise']['data']
            index_wise_data = [int(e)
                               for e in self.decrypt(ptbk, index_wise).split(",")]
            tmp["wise"] = index_wise_data
            result[name] = tmp
        return result

    def GetIndex(self, keys, start=None, end=None):
        today = date.today()
        if start is None:
            start = str(today - timedelta(days=8))
        if end is None:
            end = str(today - timedelta(days=2))

        try:
            raw_data = self.get_index_data_json(keys=keys, start=start, end=end)
            raw_data = pd.DataFrame(raw_data[keys[0]])
            raw_data.index = pd.date_range(start=start, end=end)

        except Exception as e:
            print(e)
            raw_data = pd.DataFrame({'all': [], 'pc': [], 'wise': []})

        finally:
            return raw_data

In [33]:
cookie = 'BIDUPSID=38C66F8387CAA35ED3E17BE1CBA257E9; PSTM=1678438349; BAIDUID=BF733F4FCEF20FB1E09DE932CAA5694B:FG=1; BDUSS=lJzdFRvfng5bGd5LUZpU3ltRTEtSUtmYmwyV1FHZ1ZCUjh2bFFJZVBDZS1vREprSVFBQUFBJCQAAAAAAQAAAAEAAACVYDkmcGx1c3ZyZW4AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAL4TC2S-EwtkS; newlogin=1; ZFY=WCUJvtY2X8jxAVi8Ix7hH5:Bp:B6xU0hB:BG5rvhgop7jY:C; BAIDUID_BFESS=BF733F4FCEF20FB1E09DE932CAA5694B:FG=1; BDRCVFR[vMs5fEgbCgm]=mk3SLVN4HKm; delPer=0; PSINO=3; H_PS_PSSID=; BA_HECTOR=048k0l252l8121a0a1ak8kfh1i5h7cg1n; BDORZ=FFFB88E999055A3F8A630C64834BD6D0; Hm_lvt_d101ea4d2a5c67dab98251f0b5de24dc=1683529299; bdindexid=kjigaqdq0sc48tso7ggstmsic4; SIGNIN_UC=70a2711cf1d3d9b1a82d2f87d633bd8a04335417488DOdr/Izw3Wy32J/cVXQhXAoe4/0qDiEqgLQWfBYVm0140M+cdcz9WyayYvDYD2H8NdRUE9fr4mokBzEh/RCbvAcfXx2TBIL2FhooysFUS0XMSQOheZBPQNqcAJ6yfqs8jOD8kRT0Gti8TOvMvRrABaXzSAAkWzCb+TQzUXEIfte0nzQrmYBvkjnCVpE6PIR+DdCitZ3I1tcrvNogjjug7wrYyjbIM0ubY7FSI96Llh3vk2FErneGYrRMmpo1zks8zHnX86dhZrgqkWYymZROaQ==75388531573639925584245144067641; __cas__rn__=433541748; __cas__st__212=a35a5baf16967e1ae1a1960864aedb4ad95ac3c1311a8a250efea5285aff57ed524ddd168f741ae58e25e3cd; __cas__id__212=42142278; CPID_212=42142278; CPTK_212=707047570; RT="z=1&dm=baidu.com&si=22a141fd-e50d-41ba-8dd6-b975a81597e3&ss=lhehvk23&sl=b&tt=9co&bcn=https://fclog.baidu.com/log/weirwood?type=perf"; Hm_lpvt_d101ea4d2a5c67dab98251f0b5de24dc=1683529431; BDUSS_BFESS=lJzdFRvfng5bGd5LUZpU3ltRTEtSUtmYmwyV1FHZ1ZCUjh2bFFJZVBDZS1vREprSVFBQUFBJCQAAAAAAQAAAAEAAACVYDkmcGx1c3ZyZW4AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAL4TC2S-EwtkS; ab_sr=1.0.1_Zjk5OWJjMTQzMzY4YjVkYWM0MzRlZjI2ZWZmZDg2MTUzMzM0ZjI1MjdlZjA5MzAyMmViMjNhZGUyYjVmNDc2NjUyMjkzZjY1Mjg4MWFiODMwMDRiNGUxOTkwMTM1OWU4OWY3MmRiNTVmMzlkMzNlNDAzMmJiMWNiZmIzMTFiNjE1ZDY4NjcxMGI3YjJkZjA2ZTE1ODg3ZmU5ZGY2MjhmZA=='   

# 初始化一个类
downloadbaiduindex = DownloadBaiDuIndex(cookie=cookie)

data = downloadbaiduindex.GetIndex(keys=['vr'], start='2022-04-07', end='2023-04-07')

data

[[{"name":"vr","wordType":1}]] 2022-04-07 2023-04-07


,all,pc,wise
2022-04-07,3083,988,2095
2022-04-08,3018,958,2060
2022-04-09,3394,875,2519
2022-04-10,3024,857,2167
2022-04-11,3047,986,2061
...,...,...,...
2023-04-03,2376,858,1518
2023-04-04,2312,846,1466
2023-04-05,2081,526,1555
2023-04-06,2294,865,1429


In [ ]:
data.to_('VR.csv')